# Datum und Uhrzeit

Mit pandas könnt ihr `Series` mit Datums- und Zeitinformationen erstellen. Im folgenden werden wir häufige Operationen mit Datumsdaten zeigen.

<div class="alert alert-block alert-info">

**Hinweis:**

pandas unterstützt Datumsangaben, die in [UTC](https://de.wikipedia.org/wiki/Koordinierte_Weltzeit)-Werten gespeichert sind und den Datentyp `datetime64[ns]` verwenden. Es werden auch lokale Zeiten aus einer einzigen Zeitzone unterstützt. Mehrere Zeitzonen werden durch ein [pandas.Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html)-Objekt unterstützt. Wenn ihr mit Zeiten aus mehreren Zeitzonen hantieren müsst, würde ich die Daten vermutlich nach Zeitzonen aufteilen und für jede Zeitzone einen eigenen `DataFrame` oder eine eigene `Series` verwenden.
</div>
<div class="alert alert-block alert-info">

**Siehe auch:**

* [Time series / date functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)
</div>

## Laden von UTC-Zeitdaten

In [1]:
import pandas as pd

dt = pd.date_range("2022-03-27", periods=6, freq="H")

dt

DatetimeIndex(['2022-03-27 00:00:00', '2022-03-27 01:00:00',
               '2022-03-27 02:00:00', '2022-03-27 03:00:00',
               '2022-03-27 04:00:00', '2022-03-27 05:00:00'],
              dtype='datetime64[ns]', freq='H')

In [2]:
utc = pd.to_datetime(dt, utc=True)

utc

DatetimeIndex(['2022-03-27 00:00:00+00:00', '2022-03-27 01:00:00+00:00',
               '2022-03-27 02:00:00+00:00', '2022-03-27 03:00:00+00:00',
               '2022-03-27 04:00:00+00:00', '2022-03-27 05:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='H')

<div class="alert alert-block alert-info">

**Hinweis:**

Der Typ des Ergebnisses `dtype='datetime64[ns, UTC]'` zeigt an, dass die Daten als UTC gespeichert sind.
</div>

Konvertieren wir diese Reihe in die Zeitzone Europe/Berlin:

In [3]:
utc.tz_convert('Europe/Berlin')

DatetimeIndex(['2022-03-27 01:00:00+01:00', '2022-03-27 03:00:00+02:00',
               '2022-03-27 04:00:00+02:00', '2022-03-27 05:00:00+02:00',
               '2022-03-27 06:00:00+02:00', '2022-03-27 07:00:00+02:00'],
              dtype='datetime64[ns, Europe/Berlin]', freq='H')

## Umrechnung der Ortszeit in UTC

In [4]:
local = utc.tz_convert('Europe/Berlin')

local.tz_convert('UTC')

DatetimeIndex(['2022-03-27 00:00:00+00:00', '2022-03-27 01:00:00+00:00',
               '2022-03-27 02:00:00+00:00', '2022-03-27 03:00:00+00:00',
               '2022-03-27 04:00:00+00:00', '2022-03-27 05:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='H')

## Umrechnung in Unixzeit

Wenn ihr eine `Series` mit UTC- oder Ortszeitinformationen habt, könnt ihr mit diesem Code die Sekunden nach der Unixzeit ermitteln:

In [5]:
uts = pd.to_datetime(dt).view(int) / 10**9

uts

array([1.6483392e+09, 1.6483428e+09, 1.6483464e+09, 1.6483500e+09,
       1.6483536e+09, 1.6483572e+09])

Um die Unixzeit in UTC zu laden, könnt ihr wie folgt vorgehen:

In [6]:
(pd.to_datetime(uts, unit='s').tz_localize('UTC'))

DatetimeIndex(['2022-03-27 00:00:00+00:00', '2022-03-27 01:00:00+00:00',
               '2022-03-27 02:00:00+00:00', '2022-03-27 03:00:00+00:00',
               '2022-03-27 04:00:00+00:00', '2022-03-27 05:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

## Manipulation von Terminen

### Umwandeln in Strings

Mit [pandas.DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) habt ihr einige Möglichkeiten, Datum und Uhrzeit in Strings umzuwandeln, z.B. in den Namen des Wochentags:

In [7]:
local.day_name(locale='de_DE.UTF-8')

Index(['Sonntag', 'Sonntag', 'Sonntag', 'Sonntag', 'Sonntag', 'Sonntag'], dtype='object')

Welche `locale` bei euch zur Verfügung stehen, könnt ihr euch mit `locale -a` anzeigen lassen:

In [8]:
!locale -a

C
C.UTF-8
de_AT.utf8
de_BE.utf8
de_CH.utf8
de_DE.utf8
de_IT.utf8
de_LI.utf8
de_LU.utf8
en_AG
en_AG.utf8
en_AU.utf8
en_BW.utf8
en_CA.utf8
en_DK.utf8
en_GB.utf8
en_HK.utf8
en_IE.utf8
en_IL
en_IL.utf8
en_IN
en_IN.utf8
en_NG
en_NG.utf8
en_NZ.utf8
en_PH.utf8
en_SG.utf8
en_US.utf8
en_ZA.utf8
en_ZM
en_ZM.utf8
en_ZW.utf8
POSIX


Weitere Attribute von `DatetimeIndex`, mit denen sich Datum und Zeit in Strings umwandeln lassen, sind:

Attribut | Beschreibung
:------- | :-----------
`year` | das Jahr der `datetime`
`month` | der Monat als Januar `1` und Dezember `12`
`day` | der Tag der `datetime`
`hour` | die Stunden der `datetime`
`minute` | die Minuten der `datetime`
`second` | die Sekunden der `datetime`
`microsecond` | die Mikrosekunden der `datetime`
`nanosecond` | die Nanosekunden von  `datetime`
`date` | gibt ein Numpy-Array von Python `datetime.date`-Objekten zurück
`time` | gibt ein NumPy-Array von `datetime.time`-Objekten zurück
`timetz` | liefert ein numpy-Array von `datetime.time`-Objekten mit Zeitzoneninformationen
`dayofyear`, `day_of_year` | der ordinale Tag des Jahres
`dayofweek` | der Tag der Woche mit Montag (`0`) und Sonntag (`6`)
`day_of_week` | der Tag der Woche mit Montag (`0`) und Sonntag (`6`)
`weekday` | der Tag der Woche mit Montag (`0`) und Sonntag (`6`)
`quarter` | gibt das Jahresquartal zurück
`tz` | gibt die Zeitzone zurück
`freq` | gibt das Frequenzobjekt zurück, wenn es gesetzt ist, andernfalls `None`
`freqstr` | gibt das Frequenz-Objekt als String zurück, wenn es gesetzt ist, andernfalls `None`
`is_month_start` | zeigt an, ob das Datum der erste Tag des Monats ist
`is_month_end` | zeigt an, ob das Datum der letzte Tag des Monats ist
`is_quarter_start` | zeigt an, ob das Datum der erste Tag eines Quartals ist
`is_quarter_end` | zeigt an, ob das Datum der letzte Tag eines Quartals ist
`is_year_start` | zeigt an, ob das Datum der erste Tag eines Jahres ist
`is_year_end` | zeigt an, ob das Datum der letzte Tag eines Jahres ist
`is_leap_year` | Boolescher Indikator, ob das Datum in ein Schaltjahr fällt
`inferred_freq` | versucht, eine Zeichenkette zurückzugeben, die eine durch `infer_freq` ermittelte Frequenz darstellt

Es gibt jedoch auch einige Methoden, mit denen ihr den `DatetimeIndex` in Stringsumwandeln könnt, z.B. `strftime`:

In [9]:
local.strftime('%d.%m.%Y')

Index(['27.03.2022', '27.03.2022', '27.03.2022', '27.03.2022', '27.03.2022',
       '27.03.2022'],
      dtype='object')

<div class="alert alert-block alert-info">

**Hinweis:**

In [strftime() and strptime() Format Codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes) erhaltet ihr eine Übersicht über die verschiedenen Formattierungsmöglichkeiten von `strftime`.
</div>

Weitere Methoden sind:

Methode | Beschreibung
:------ | :-----------
`normalize` | konvertiert Zeiten in Mitternacht
`strftime` | konvertiert in Index unter Verwendung des angegebenen Datumsformats
`snap` | rastet den Zeitstempel auf der nächsten auftretenden Frequenz ein
`tz_convert` | konvertiert ein `tz`-fähiges Datetime-Array/Index von einer Zeitzone in eine andere
`tz_localize` | Lokalisiert `tz`-naives Datetime Array/Index in `tz`-kompatibles Datetime Array/Index
`round` | rundet die Daten zur nächsten angegebenen Frequenz
`floor` | rundet die Daten ab auf die angegebene Frequenz
`ceil` | rundet die Daten auf auf die angegebene Frequenz
`to_period` | wandelt die Daten um in einen PeriodArray/Index bei einer bestimmten Frequenz
`to_perioddelta` | Berechnet `TimedeltaArray` der Differenz zwischen den Indexwerten und dem in `PeriodArray` umgewandelten Index bei der angegebenen Frequenz
`to_pydatetime` | gibt `Datetime` Array/Index als `ndarray`-Objekt von `datetime.datetime`-Objekten zurück
`to_series` | erzeugt eine `Series` mit Index und Werten, die den Indexschlüsseln entsprechen; nützlich mit `map` für die Rückgabe eines Indexers
`to_frame` | erzeugt einen `DataFrame` mit einer Spalte, die den Index enthält
`month_name` | gibt die Monatsnamen des `DateTimeIndex` mit dem angegebenen `locale` zurück
`day_name` | gibt die Tagesnamen des `DateTimeIndex` mit dem angegebenen `locale`  zurück
`mean` | gibt den Mittelwert des Arrays zurück
`std` | gibt die Standardabweichung der Stichprobe über die angeforderte Achse zurück